In [502]:
#importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [503]:
data=pd.read_csv('cpdata.csv')
print(data.head(1))

   temperature   humidity    rainfall label
0    20.879744  82.002744  202.935536  rice


In [504]:
#Creating dummy variable for target i.e label
label= pd.get_dummies(data.label).iloc[: , 1:]
data= pd.concat([data,label],axis=1)
data.drop('label', axis=1,inplace=True)
print('The data present in one row of the dataset is')
print(data.head(2))
column_headers = list(data.columns.values)
print("The Column Header :", column_headers)
train=data.iloc[:, 0:3].values
test=data.iloc[: ,3:].values

The data present in one row of the dataset is
   temperature   humidity    rainfall  Black gram  Chickpea  Coconut  Coffee  \
0    20.879744  82.002744  202.935536           0         0        0       0   
1    21.770462  80.319644  226.655537           0         0        0       0   

   Cotton  Ground Nut  Jute  ...  maize  mango  millet  muskmelon  orange  \
0       0           0     0  ...      0      0       0          0       0   
1       0           0     0  ...      0      0       0          0       0   

   papaya  pomegranate  rice  watermelon  wheat  
0       0            0     1           0      0  
1       0            0     1           0      0  

[2 rows x 33 columns]
The Column Header : ['temperature', 'humidity', 'rainfall', 'Black gram', 'Chickpea', 'Coconut', 'Coffee', 'Cotton', 'Ground Nut', 'Jute', 'Kidney Beans', 'Lentil', 'Moth Beans', 'Mung Bean', 'Peas', 'Pigeon Peas', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'apple', 'banana', 'grapes', 'maize', 'mango', 'mill

In [505]:
#Dividing the data into training and test set
X_train,X_test,y_train,y_test=train_test_split(train,test,test_size=0.1)
print(X_train.shape)
print(X_test[0])
print(y_train.shape)
print(y_test[0])

(2790, 3)
[30.10773379 90.34546355 75.24521981]
(2790, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


In [506]:
def init_var():
    w1 = np.random.rand(30, 3) - 0.5
    b1 = np.random.rand(30, 1) - 0.5
    w2 = np.random.rand(30, 30) - 0.5
    b2 = np.random.rand(30, 1) - 0.5
    return w1, b1, w2, b2

In [507]:
def ReLU(z):
    return np.maximum(z, 0)

def Softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def derv_ReLU(z):
    return z > 0

In [508]:
# arr is a 2d numpy array, assign the maximum value in each row to 1 and the rest to 0
def max_to_one(arr, m, n):
    for i in range(0, m):
        d = np.argmax(arr[i])
        for j in range(0, n):
            if(d == j):
                arr[i][j] = 1
            else:
                arr[i][j] = 0
    return arr

In [509]:
def forward_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = Softmax(z2)
    a2 = a2.T
    #find the number of rows in a2
    m = a2.shape[0]
    a2 = max_to_one(a2, m, 30)
    # print(a2)
    # print(a2.shape)
    return z1, a1, z2, a2

def back_prop(w1, b1, w2, b2, x, y, z1, a1, z2, a2):
    m = y.size
    dz2 = (a2 - y).T
    dw2 = 1 / m * (dz2.dot(a1.T))
    db2 = 1 / m * np.sum(dz2)
    dz1 = w2.T.dot(dz2) * derv_ReLU(z1)
    dw1 = 1 / m * (dz1.dot(x.T))
    db1 = 1 / m * np.sum(dz1)
    return dw1, db1, dw2, db2

def update_var(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate):
    w1 = w1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1
    w2 = w2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
    return w1, b1, w2, b2

In [510]:
def accuracy(y_pred, y):
    return np.mean(y_pred == y)

def grad_descent(x, y, alpha, w1, b1, w2, b2):
    z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
    dw1, db1, dw2, db2 = back_prop(w1, b1, w2, b2, x, y, z1, a1, z2, a2)
    w1, b1, w2, b2 = update_var(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
    return w1, b1, w2, b2, a2

In [511]:
from sklearn.utils import shuffle
batch_size = 32
n_example = 2790
batches = int(n_example/batch_size)
epochs = 100
w1, b1, w2, b2 = init_var()
acc = np.empty(0)
for epoch in range(epochs):
    i = shuffle(range(n_example))
    for b in range(batches):
        x_batch = X_train[i[b*batch_size:(b+1)*batch_size]]
        y_batch = y_train[i[b*batch_size:(b+1)*batch_size]]
        w1, b1, w2, b2, a2 = grad_descent(x_batch.T, y_batch, 0.1, w1, b1, w2, b2)
        acc = np.append(acc, accuracy(a2, y_batch))
    print("Iteration: ", epoch)
    print("Accuracy: ", np.mean(acc))

Iteration:  0
Accuracy:  0.9382902298850574
Iteration:  1
Accuracy:  0.9391642720306513
Iteration:  2
Accuracy:  0.9395035121328225
Iteration:  3
Accuracy:  0.9397090517241378
Iteration:  4
Accuracy:  0.939875478927203
Iteration:  5
Accuracy:  0.9399664750957855
Iteration:  6
Accuracy:  0.9400348932676518
Iteration:  7
Accuracy:  0.9400951867816093
Iteration:  8
Accuracy:  0.9401367603235419
Iteration:  9
Accuracy:  0.9401592432950191
Iteration:  10
Accuracy:  0.9401830808080809
Iteration:  11
Accuracy:  0.9402229007024265
Iteration:  12
Accuracy:  0.94024738432066
Iteration:  13
Accuracy:  0.9402717911877393
Iteration:  14
Accuracy:  0.9402961366538952
Iteration:  15
Accuracy:  0.9403174389367817
Iteration:  16
Accuracy:  0.9403348264593193
Iteration:  17
Accuracy:  0.9403496168582376
Iteration:  18
Accuracy:  0.9403596995361967
Iteration:  19
Accuracy:  0.9403735632183908
Iteration:  20
Accuracy:  0.9403855363984673
Iteration:  21
Accuracy:  0.9403991422849182
Iteration:  22
Accuracy

In [520]:
atemp, ah, rain = 45.3, 27.7, 71.2
l=[]
l.append(atemp)
l.append(ah)
l.append(rain)
#create a numpy arrray with atemp, ah and rain
predictcrop = np.array(l)
predictcrop = predictcrop.reshape(1, 3)
print(predictcrop)
print(predictcrop.shape)

[[45.3 27.7 71.2]]
(1, 3)


In [513]:
# Putting the names of crop in a single list
crops=['Black gram', 'Chickpea', 'Coconut', 'Coffee', 'Cotton', 'Ground Nut', 'Jute', 'Kidney Beans', 'Lentil', 'Moth Beans', 'Mung Bean', 'Peas', 'Pigeon Peas', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'apple', 'banana', 'grapes', 'maize', 'mango', 'millet', 'muskmelon', 'orange', 'papaya', 'pomegranate', 'rice', 'watermelon', 'wheat']
print(len(crops))

30


In [514]:
def pred(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = Softmax(z2)
    a2 = a2.T
    #a2 = max_to_one(a2, 30, 30)
    return a2

In [529]:
# print(w1.shape)
# print(b1.shape)
# print(w2.shape)
# print(b2.shape)
# print(predictcrop.shape)
# x1 = w1
# x2 = b1
# x3 = w2
# x4 = b2
print(predictcrop)
a2 = pred(w1, b1, w2, b2, predictcrop.T)
#print(a2.shape)
#print(a2)
#find mean of each column of a2
#a2 = np.mean(a2, axis=0)
#find maximum in a2
# d = np.argmax(a2[0])
# #find the index of the second largest number in a2
# d2 = np.argpartition(a2.flatten(), -2)[-2]
# print("The crop recommended is ",crops[d])
# print(d)
# print("The second crop recommended is ",crops[d2])
# print(d2)
# print(a2)
thres = 0.07
df = np.argwhere(a2 > thres)
# print(df)
print("Recommended crops are: ")
if not df.size:
    print("Null")
for i in df:
    print(crops[i[1]])

[[45.3 27.7 71.2]]
[[0.06139362 0.02264354 0.00481489 0.00942125 0.00696515 0.07506778
  0.00912583 0.02536149 0.01633189 0.06128055 0.08035828 0.07338618
  0.05735041 0.02037374 0.02427284 0.02958897 0.0351934  0.02803672
  0.02734176 0.03334661 0.00942459 0.0158971  0.04270099 0.01692395
  0.03578801 0.00909738 0.05692328 0.06074951 0.02302301 0.02781725]]
Recommended crops are: 
Ground Nut
Mung Bean
Peas
